In [ ]:
!pip install MetaTrader5 pandas pyarrow fastparquet

### this is the notebook for messing around to see what i find in the datta

In [5]:
# establishing a connection

import MetaTrader5 as mt5

# 1. Define the path to the FBS EXE file

mt5_path = r"C:\Program Files\FBS MetaTrader 5\terminal64.exe"


print(f"Attempting to connect to: {mt5_path}...")

# 2. Initialize the connection
# We pass the path so Python doesn't look for the generic version
if not mt5.initialize(path=mt5_path):
    print("❌ Connection Failed")
    print("Error Code:", mt5.last_error())
else:
    print("✅ Connection Successful!")
    
    # 3. Print Account Info to prove we are talking to the broker
    account_info = mt5.account_info()
    if account_info:
        print(f"Connected to Account: {account_info.login}")
        # print(f"Connected to Account: {account_info.password}")
        print(f"Server: {account_info.server}")
        print(f"Balance: {account_info.balance} USD")
    
    # 4. Close connection
    mt5.shutdown()

Attempting to connect to: C:\Program Files\FBS MetaTrader 5\terminal64.exe...
✅ Connection Successful!
Connected to Account: 105395845
Server: FBS-Demo
Balance: 20000.0 USD
